In [ ]:
import pandas as pd
import numpy as np

from src.all_in_one import *

In [ ]:
processed_data_path = path_manager.processed_data

In [ ]:
referral_id = pd.DataFrame()
if os.path.exists(processed_data_path) and os.path.isdir(processed_data_path):
    for file_name in os.listdir(processed_data_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(processed_data_path, file_name)
            df = pd.read_csv(file_path)
            df_rid = df['rid']
            referral_id = pd.concat([referral_id, df_rid])

In [ ]:
referral_id = referral_id.drop_duplicates(keep='first')
referral_id = referral_id.sort_values(by='rid', ascending=True).reset_index(drop=True)

In [ ]:
scr_file = path_manager.get_data_file(Category.PS, PSFile.SCR)
scr_dataset = Dataset(config_file, scr_file.sheet)
scr = get_df(scr_file.path, scr_file.sheet, config_file)

In [ ]:
service_dataset = pd.merge(referral_id, scr, on='rid', how='left')

In [ ]:
# service_dataset

In [ ]:
scr_rid = scr[['rid', 'sc_s1']]
scr_rid['is_duplicated'] = scr_rid['rid'].duplicated(keep=False)
scr_rid['is_NaT'] = scr_rid['sc_s1'].isna()

In [ ]:
conditions = [
    (scr_rid['is_duplicated'] & ~scr_rid['is_NaT']),
    (scr_rid['is_duplicated'] & scr_rid['is_NaT']),
    (~scr_rid['is_duplicated'])
]

choices = [1, 2, 0]

scr_rid['duplicated'] = np.select(conditions, choices, default=2)

In [ ]:
scr_rid